# Introducción a la Ciencia de Datos: Tarea 1

Este notebook contiene el código de base para realizar la Tarea 1 del curso. Puede copiarlo en su propio repositorio y trabajar sobre el mismo.
Las **instrucciones para ejecutar el notebook** están en la [página inicial del repositorio](https://gitlab.fing.edu.uy/maestria-cdaa/intro-cd).

Se utiliza el lenguaje Python y la librería Pandas. Si no tiene ninguna familiaridad con la librería, se recomienda realizar algún tutorial introductorio (ver debajo).
También se espera que los alumnos sean proactivos a la hora de consultar las documentaciones de las librerías y del lenguaje, para entender el código provisto.
Además de los recursos provistos en la [página del curso](https://eva.fing.edu.uy/course/view.php?id=1378&section=1), los siguientes recursos le pueden resultar interesantes:
 - [Pandas getting started](https://pandas.pydata.org/docs/getting_started/index.html#getting-started) y [10 minutes to pandas](https://pandas.pydata.org/docs/user_guide/10min.html): Son parte de la documentación en la página oficial de Pandas.
 - [Kaggle Learn](https://www.kaggle.com/learn): Incluye tutoriales de Python y Pandas.


Si desea utilizar el lenguaje R y está dispuesto a no utilizar (o traducir) este código de base, también puede hacerlo.

En cualquier caso, **se espera que no sea necesario revisar el código para corregir la tarea**, ya que todos los resultados y análisis relevantes deberían estar en el **informe en formato PDF**.

## Cargar bibliotecas (dependencias)
Recuerde instalar los requerimientos (`requirements.txt`) en el mismo entorno donde está ejecutando este notebook (ver [README](https://github.com/DonBraulio/introCD)).

In [360]:
from time import time
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

# Agregue aqui el resto de las librerias que necesite
# from ...
# import ...

## Lectura de Datos

In [361]:
# DataFrame con todos los discursos:
df_speeches = pd.read_csv('../data/us_2020_election_speeches.csv')
df_speeches

,speaker,title,text,date,location,type
0,David Perdue,Georgia Sen. David Perdue Speech Transcript at...,David Perdue: (00:01)\nHow great is it to be b...,"Oct 16, 2020","Macon, Georgia",Campaign Speech
1,Joe Biden,"Joe Biden Southfield, MI Speech on Health Care...","Joe Biden: (00:00)\nHello, Michigan. Hi, how a...","Oct 16, 2020","Southfield ,Michigan",Campaign Speech
2,Donald Trump,Donald Trump Speech Transcript ‘Protecting Ame...,President Trump: (00:30)\nThank you. What a ni...,"Oct 16, 2020","Fort Myers, Florida",Campaign Speech
3,Joe Biden,Joe Biden ABC Town Hall Transcript October 15,"George Stephanopoulos: (00:41)\nHey, and welco...","Oct 15, 2020",ABC,Town Hall
4,Donald Trump,Donald Trump NBC Town Hall Transcript October 15,Savannah Guthrie: (03:50)\nIt’s nothing but no...,"Oct 15, 2020",NBC,Town Hall
...,...,...,...,...,...,...
264,Bernie Sanders,Bernie Sanders Speech Transcript: Sanders Spea...,Bernie Sanders: (00:00)\nJust want to take thi...,"Feb 6, 2020",Iowa,Campaign Speech
265,Democratic Candidates,Transcript: Speeches at the Iowa Caucuses – Be...,Bernie Sanders: (00:08)\nThank you. Thank you....,"Feb 4, 2020",Iowa,Campaign Speech
266,Donald Trump,Donal Trump Iowa Rally Transcript: Trump Holds...,Donald Trump: (00:24)\nI worked so hard for th...,"Jan 30, 2020","Des Moines, Iowa",Campaign Speech
267,Donald Trump,Donald Trump New Jersey Rally Speech Transcrip...,Donald Trump: (01:22)\nThank you. Thank you. I...,"Jan 28, 2020","Wildwood, New Jersey",Campaign Speech


# Parte 1: Cargado y Limpieza de Datos

## Exploración de Datos

In [362]:
# TODO: Analice la cantidad de discursos por candidato

#1-A) ANALISIS DE CALIDAD DE DATOS 
# Reporte si existen datos faltantes en algún campo, o cualquier otro problema de calidad de datos que encuentre.
# i) Valores faltantes
print(df_speeches.info())
print(df_speeches.isna().sum())

# ii) Registros duplicados 
print(df_speeches.duplicated().sum())
print(df_speeches[df_speeches.duplicated(keep=False)].head())

# iii) Consistencia en el formato de las fechas; las no parseables serán NaT
df_speeches['date_parsed'] = pd.to_datetime(df_speeches['date'], errors='coerce')
# Contar cuántas fechas fallaron al parsear
print(df_speeches['date_parsed'].isna().sum())
# Actualidad de los Datos. Rango de fechas parseadas
print(df_speeches['date_parsed'].min(), df_speeches['date_parsed'].max())

# iv) Contar discursos que contienen al menos un carácter fuera del rango ASCII
mask = df_speeches['text'].str.contains(r'[^\x00-\x7F]', na=False)
mask.sum()
# Mostrar ejemplos de esos discursos
print(df_speeches[mask].head())

# v) Inconsistencias en categorías (mayúsculas/minúsculas, variaciones)
print(df_speeches['speaker'].value_counts(dropna=False))
# Valores únicos tras unificar a minúsculas
df_speeches['speaker'].str.lower().value_counts().head(10)

# vi) Verifico que los candidatos de Speaker aparecen en el Texto 
# a) Hallo los candidatos que aparecen como oradores en la columna texto
import re

# ORADORES DE LA COLUMNA SPEAKER
# 1) Separar cada valor de 'speaker' por coma en columnas
DS_orador_from_speaker = df_speeches['speaker'].str.split(',', expand=True)

# 2) Normalizar: pasar a minúsculas y eliminar espacios al inicio
DS_orador_from_speaker = DS_orador_from_speaker.applymap(
    lambda x: x.lower().lstrip() if isinstance(x, str) else x
)
print("HOLA")
print(DS_orador_from_speaker)

# 1) Apilar todas las columnas en una Serie, quitar NaN y espacios
serie = DS_orador_from_speaker.stack()              # apila columnas a índice :contentReference[oaicite:0]{index=0}
serie = serie.dropna().str.strip()                  # elimina NaN y espacios en extremos

# 2) Extraer valores únicos, manteniendo el orden original
valores_unicos = serie.unique()                     # devuelve ndarray con valores únicos :contentReference[oaicite:1]{index=1}

# 3) Crear el DataFrame de una sola columna
DF_lista_oradores_from_Speaker = pd.DataFrame(
    valores_unicos,
    columns=['speaker']
)
print(f"Lista de oradores: \n{DF_lista_oradores_from_Speaker}")


# ORADORES DE LA COLUMNA TEXT
# 1) Definimos un patrón que busca:
#    • comienzo de texto (^) o bien un signo de puntuación (. : ? ") seguido opcionalmente de espacio
#    • luego un nombre propio con mayúscula inicial (incluye tildes y Ñ), posiblemente compuesto
#    • justo antes de “: (min:seg)”
patron = re.compile(
    r'(?:^|[\.:\?"\!]\s*)'                              # inicio o signo de puntuación + espacio
    r'([A-ZÁÉÍÓÚÑ][\wáéíóúñ]+(?:\s+[A-ZÁÉÍÓÚÑ][\wáéíóúñ]+)*)'  # nombre(s) propio(s)
    r'\s*:\s*\(\d{1,2}:\d{2}\)'                         # luego  : (min:seg)
)

def extraer_oradores(texto):
    """
    Devuelve una lista con todos los nombres que coinciden en el texto dado.
    """
    if not isinstance(texto, str):
        return []
    # re.findall devuelve la lista de grupos 1 (los nombres)
    return patron.findall(texto)

# 2) Aplicamos la extracción a cada fila de df_speakers['text']
serie_listas = df_speakers['text'].apply(extraer_oradores)

# 3) Averiguamos cuántos oradores como máximo salen en alguna fila
max_oradores = serie_listas.map(len).max()

# 4) Creamos un nuevo DataFrame donde cada columna será speaker_1, speaker_2, …
columnas = [f"speaker_{i+1}" for i in range(max_oradores)]
df_oradores_en_text = pd.DataFrame(
    serie_listas.tolist(),
    index=df_speakers.index,
    columns=columnas
)

print(df_oradores_en_text)



# 1) Normalizar valores: quitar espacios al inicio/final y pasar todo a minúsculas
DS_orador_from_text = df_oradores_en_text.applymap(
    lambda x: x.strip().lower() if isinstance(x, str) else x
)

# 2) Aplanar todas las columnas en una única serie, eliminar nulos y duplicados
serie_todos = DS_orador_from_text.stack().reset_index(drop=True)
serie_unicos = serie_todos.dropna().drop_duplicates().reset_index(drop=True)

# 3) Construir el DataFrame final de una sola columna
DS_lista_orador_from_text = serie_unicos.to_frame(name='speaker')

# Opcional: echar un vistazo
#print(DS_orador_from_text.to_string())
#print(DS_lista_orador_from_text.to_string())




# 1) Contar frecuencia de cada orador
frecuencias = DS_orador_from_text.stack().value_counts()

# 2) Añadir la columna 'count' a tu DataFrame de oradores únicos
DS_lista_orador_from_text['count'] = DS_lista_orador_from_text['speaker'].map(frecuencias)

# 3) Ordenar de mayor a menor según la frecuencia
DS_lista_orador_from_text = (
    DS_lista_orador_from_text
      .sort_values(by='count', ascending=False)
      .reset_index(drop=True)
)

# 4) (Opcional) Si no quieres mantener la columna 'count', la eliminas:
# DS_lista_orador_from_text = DS_lista_orador_from_text.drop(columns='count')

print(DS_lista_orador_from_text.to_string())









# En particular, analice la cantidad de discursos por candidato/a, y a partir de este punto trabaje con los cinco candidatos/as con mayor cantidad de discursos.


DF2 = df_speeches['speaker'].str.split(',', expand=True)
DF3 = DF2.map(
    lambda x: x.lower().lstrip() if isinstance(x, str) else x,
    na_action="ignore"
)
conteo_oradores = DF3.stack().value_counts()
print(conteo_oradores)


top_5_candidatos = conteo_oradores.head(5).index.tolist()

print(df_top5)



# df_speeches_top_5 = ...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 269 entries, 0 to 268
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   speaker   266 non-null    object
 1   title     269 non-null    object
 2   text      269 non-null    object
 3   date      269 non-null    object
 4   location  251 non-null    object
 5   type      248 non-null    object
dtypes: object(6)
memory usage: 12.7+ KB
None
speaker      3
title        0
text         0
date         0
location    18
type        21
dtype: int64
0
Empty DataFrame
Columns: [speaker, title, text, date, location, type]
Index: []
0
2020-01-15 00:00:00 2020-10-16 00:00:00
        speaker                                              title  \
0  David Perdue  Georgia Sen. David Perdue Speech Transcript at...   
1     Joe Biden  Joe Biden Southfield, MI Speech on Health Care...   
2  Donald Trump  Donald Trump Speech Transcript ‘Protecting Ame...   
3     Joe Biden      Joe Biden ABC Town

/var/folders/wd/m0sc8txj5jlf18crlj8_1p7r0000gn/T/ipykernel_1545/2425610811.py:40: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  DS_orador_from_speaker = DS_orador_from_speaker.applymap(


                 speaker_1         speaker_2              speaker_3  \
0             David Perdue      David Perdue           David Perdue   
1                Joe Biden         Joe Biden              Joe Biden   
2          President Trump             Crowd        President Trump   
3    George Stephanopoulos         Joe Biden  George Stephanopoulos   
4         Savannah Guthrie  Savannah Guthrie        President Trump   
..                     ...               ...                    ...   
264         Bernie Sanders    Bernie Sanders         Bernie Sanders   
265         Bernie Sanders    Bernie Sanders         Bernie Sanders   
266           Donald Trump      Donald Trump           Donald Trump   
267           Donald Trump      Donald Trump           Donald Trump   
268           Wolf Blitzer    Bernie Sanders         Bernie Sanders   

            speaker_4              speaker_5         speaker_6  \
0        David Perdue           David Perdue      David Perdue   
1           Joe

/var/folders/wd/m0sc8txj5jlf18crlj8_1p7r0000gn/T/ipykernel_1545/2425610811.py:100: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  DS_orador_from_text = df_oradores_en_text.applymap(


## Visualizaciones

In [363]:
# TODO: Visualización de los discursos de cada candidato a lo largo del tiempo 


## Limpieza de Texto y Conteo de Palabras

In [364]:
def clean_text(df, column_name):

    # Eliminar primeras palabras hasta el primer "\n"
    result = df[column_name].str.replace(r"^[^\n]*\n", "", regex=True)

    # Convertir todo a minúsculas
    result = result.str.lower()

    # TODO: completar signos de puntuación faltantes
    for punc in ["[", "\n", ",", ":", "?"]:
        result = result.str.replace(punc, " ")
    
    return result

# TODO: Creamos una nueva columna CleanText a partir de text
# df_speeches_top_5["CleanText"] = 

In [365]:
# Convierte párrafos en listas "palabra1 palabra2 palabra3" -> ["palabra1", "palabra2", "palabra3"]
df_speeches_top_5["WordList"] = df_speeches_top_5["CleanText"].str.split()

# Veamos la nueva columna creada: notar que a la derecha tenemos una lista: [palabra1, palabra2, palabra3]
df_speeches_top_5[["CleanText", "WordList"]]

NameError: name 'df_speeches_top_5' is not defined

# Parte 2: Conteo de Palabras y Visualizaciones

 ## Candidatos con mayor cantidad de palabras

In [ ]:
# TODO: Realice una visualización que permita comparar las palabras más frecuentes de cada uno de los cinco candidatos/as. 
# - Encuentra algún problema en los resultados?


In [ ]:
# TODO: Busque los candidatos/as con mayor cantidad de palabras.


In [ ]:
# TODO: Construya una matriz de 5x5, donde cada fila y columna corresponden a un candiato/a, 
# y la entrada (i,j) contiene la cantidad de veces que el candiato/a “i” menciona al candiato/a “j”.

# mentions_matrix = ...

# Opcional: Genere un grafo dirigido con esa matriz de adyacencia para visualizar las menciones. 
# Puede ser util la biblioteca networkx